# Frequency and power distribution comparison between 1st and 5th hour of POST 
- both measures were calculated using wavelet transform
- Runs slow, maybe adding parallel loop for wavelet calculation will help, or move to hilbert amplitude which will be lot faster.

In [ ]:
import matplotlib.pyplot as plt
from neuropy.utils.signal_process import TimeFrequency
import numpy as np
from neuropy.plotting import Fig
import pandas as pd
from scipy import stats
import subjects

sd_sessions = (
    subjects.sd.ratJday1
    + subjects.sd.ratKday1
    + subjects.sd.ratNday1
    + subjects.sd.ratSday3
    + subjects.sd.ratRday2
    + subjects.sd.ratUday4
    + subjects.sd.ratVday2
)
nsd_sessions = (
    subjects.nsd.ratJday2
    + subjects.nsd.ratKday2
    + subjects.nsd.ratNday2
    + subjects.nsd.ratSday2
    + subjects.nsd.ratRday1
    + subjects.nsd.ratUday2
    + subjects.nsd.ratVday1
)


In [ ]:
from neuropy.utils.signal_process import hilbert_ampltiude_stat

rpl_power = pd.DataFrame()
rpl_freq_dist = pd.DataFrame()
for grp, sessions in zip(["sd", "nsd"], [sd_sessions, nsd_sessions]):
    for sub, sess in enumerate(sessions):
        print(grp,sub)
        post = sess.paradigm["post"].flatten()
        # sd = sess.paradigm["sd"].flatten()
        sd = [post[0], post[0] + 5 * 3600]
        channels = sess.ripple.metadata["channels"]

        signal = sess.eegfile.get_signal(channels, 1800, 3600)
        hilbert_stat = hilbert_ampltiude_stat(
            signal.traces, freq_band=(150, 250), fs=1250, statistic="median"
        )

        channel = channels[np.argmax(hilbert_stat)]

        freqs = np.arange(130, 250, 2)
        rpls_1h = sess.ripple.time_slice(sd[0], sd[0] + 3600).set_labels("1st")
        rpls_5h = sess.ripple.time_slice(sd[0] + 4 * 3600, sd[1]).set_labels("5th")
        rpls_6h = sess.ripple.time_slice(sd[1], sd[1] + 3600).set_labels("6th")

        rpls_comb = rpls_1h + rpls_5h + rpls_6h
        max_power, freq_dist = [], []
        for start, stop in zip(rpls_comb.starts, rpls_comb.stops):
            signal = sess.eegfile.get_signal(channel, start, stop)
            wvlt = TimeFrequency(
                signal=signal, freqs=freqs, method="wavelet", ncycles=7
            )
            wvlt_mean = wvlt.mean_power()
            max_power.append(np.max(wvlt_mean))
            freq_dist.append(np.mean(wvlt.traces, axis=1))

        df1 = pd.DataFrame(
            {
                "max_power": np.array(max_power),
                "hour": rpls_comb.labels,
                "sub": sub,
                "grp": grp,
            }
        )
        df2 = pd.DataFrame(np.asarray(freq_dist), columns=freqs)
        df2["hour"] = rpls_comb.labels
        df2["sub"] = sub
        df2["grp"] = grp
        df2 = pd.melt(
            df2,
            id_vars=["hour", "sub", "grp"],
            value_vars=freqs,
            var_name=["freq"],
            value_name="freq_power",
            ignore_index=True,
        )

        rpl_power = rpl_power.append(df1).reset_index(drop=True)
        rpl_freq_dist = rpl_freq_dist.append(df2).reset_index(drop=True)


## Frequency distribution plot

In [ ]:
%matplotlib widget
import seaborn as sns

dist = sns.relplot(
    data=rpl_freq_dist,
    x="freq",
    y='freq_power',
    hue="hour",
    ci=None,
    kind='line',
    row="sub",
    col='grp',
    height=1.5,
    aspect=2,
    lw=1,
)

# dist.figure.savefig(subjects.figpath_sd/'ripple_power_dist_Zt1_vs_Zt5.pdf')

## Power distribution plot

In [ ]:
%matplotlib widget
from neuropy import plotting
import seaborn as sns

dist = sns.displot(
    data=rpl_power,
    x="max_power",
    hue="hour",
    stat="probability",
    # binrange=(0, 140),
    kind='hist',
    row="sub",
    col='grp',
    cumulative=False,
    # ax=ax,
    element="step",
    fill=False,
    common_norm=False,
    height=1.5,
    aspect=2,
    lw=1,
    # alpha=0.8,
    palette=['#5cc0eb','#faa49d','#05d69e'],
    
)

# dist.figure.savefig(subjects.figpath_sd/'ripple_power_dist_Zt1_vs_Zt5_vs_Zt6.pdf')


# Spike triggered raster plot w.r.t ripples during sleep deprivation for individual neurons

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import subjects

sessions = subjects.sd.ratSday3

In [ ]:
for sub,sess in enumerate(sessions):
    sd = sess.paradigm['sd']
    rpls = sess.ripple.time_slice(sd[0],sd[1])
    